### Installations
1. **lxml** parser is installled to proccess the html.
2. **BeautifulSoup** is intalled to manipulate the the table easier.

In [1]:
pip install lxml

     |████████████████████████████████| 5.8MB 27.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 17.1MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [28]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

### The soup is created

In [29]:
soup = BeautifulSoup(page, "lxml")

We print the soup using *'prettify'* to make it look like html and read it easier.

A list **'data'** is created to store the data from the table in form of lists.

The list is populated *through the loop*.

'tr' in the html represents new row in the table and 'td' represents new cell.

i.text is used to get *only the text* from each record and not the html format (< tr >,< td >, < head >, etc.).

Each **'row'** created is a list used to populate the **'data'** list.

In [30]:
data = []
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    
    data.append(row)

An empty dataframe is created with 3 columns: 
1. PostalCode
2. Borough 
3. Neighborhood

In [31]:
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
df

,PostalCode,Borough,Neighborhood


### Dataframe data
We store the data in the dataframe by running through the **'data'** list. The __*'len(data[i]) == 3'*__ conditon is set because at the end of the table there are some errors because of the html. 

In [32]:
for i in range (0, len(data)):
    if len(data[i]) == 3:
        df = df.append({
             "PostalCode": data[i][0],
             "Borough":  data[i][1],
            "Neighborhood": data[i][2]
              }, ignore_index=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
...,...,...,...
283,M8Z,Etobicoke,Mimico NW\n
284,M8Z,Etobicoke,The Queensway West\n
285,M8Z,Etobicoke,Royal York South West\n
286,M8Z,Etobicoke,South of Bloor\n


We **clean the data** by deleting the records which include **'Not assigned'** values in the **'Borough' column**.

Next, we reset the index of the dataframe so that it starts from zero and includes all numbers.

In [33]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West\n
207,M8Z,Etobicoke,Mimico NW\n
208,M8Z,Etobicoke,The Queensway West\n
209,M8Z,Etobicoke,Royal York South West\n


Finally, we update the Neighborhood's names with the same values excluding __\n__.

In [34]:
for index, rows in df.iterrows():
    df.at[index,'Neighborhood'] = df.at[index,'Neighborhood'].replace('\n','')
    
df.head(7)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


In [35]:
df.shape

(211, 3)

We **find** all the records that have **'Not assigned'** in the 3rd column and store their **indexes** in the **missing** list.

In [36]:
missing = []
for index, rows in df.iterrows():
    if df.at[index,'Neighborhood'] == 'Not assigned' :
        print (index, df.at[index,'Neighborhood'])
        missing.append(index)

print (missing)

6 Not assigned
[6]


We update Not Assigned Neighborhood cells with the same value from the 'Borough' column.

In [37]:
for i in range (0, len(missing)):
    print (df.loc[[missing[i]]])
    df.at[missing[i],'Neighborhood'] = df.at[missing[i], 'Borough']
    print (df.loc[[missing[i]]])

  PostalCode       Borough  Neighborhood
6        M7A  Queen's Park  Not assigned
  PostalCode       Borough  Neighborhood
6        M7A  Queen's Park  Queen's Park


## Merging records with same Postal Code.

We create a list **'indexes'** which will be populated with lists **'temp'** of the dataframe's indexes which records have the same PostalCode.

Through the nested loop we append the _'temp'_ list with the indexes where same postal codes exist. The condition _'index != index1'_ is used to to exclude comparison of the same records.

-*Set* function is used to remove duplicates from the *'temp'* list.  
-The condition *'len(temp) > 0'* is used to append *'indexes'* only with populated lists, because during the loop when there are no similar Postal codes empty lists are still created.

In [38]:
indexes = []
for index, rows in df.iterrows():
    temp = []
    
    for index1, r in df.iterrows():
        if index != index1 and df.at[index, 'PostalCode'] == df.at[index1, 'PostalCode']:
            temp.append(index)
            temp.append(index1)
    
    temp = list(set(temp))
    
    if len(temp) > 0:
        indexes.append(temp)
print ('Operation completed!')

Operation completed!


By applying the following methods we create an indexes list with unique lists in it.

In [39]:
uniq_indexes = [list(t) for t in set(map(tuple, indexes))]

The following loops:
1. Create a list **'_names_'** which will be populated with the Neighborhoods' names, which have the same Postal Code. Data are extracted from the DataFrame *names.append(df.at[x,'Neighborhood']))*
2. Create a string **'_names_'** which includes all Neighborhoods' names, that have the same postal code, separated with commas. (*', '.join(names)*)
3. **Update** the value of the Neighborhood name **on the first row** of those that have the same postal code.
4. **Drop** all the others rows with the same postal code.

In [40]:
for i in uniq_indexes:
    names = []
    for x in i:
        names.append(df.at[x,'Neighborhood'])          

    name = ', '.join(names)                          
       
    df.at[i[0],'Neighborhood'] = name
        
    for z in range (1, len(i)):
        a = i[z]
        df.drop(a, inplace=True)

In [41]:
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [42]:
df.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [43]:
print ('The dataframe has {} rows and {} columns'.format(df.shape[0],df.shape[1]))

The dataframe has 103 rows and 3 columns


In [47]:
df.to_csv('toronto_data.csv', encoding='utf-8', index=False)